<img src="https://www.amazeemetrics.com/sites/default/files/Getting-Started-with-Google-Analytics.jpg">

<a href='#'>Preface</a><br>
<a href='#desc'>description</a><br>
<a href='#about'>About notebook</a><br>
<a href='#load_lib'>Load libraries</a><br>
<a href='#load_data'>Load Dataset</a><br>
<a href='#description'>Column Description</a><br>
<a href='#cleaning'>Cleaning Dataset</a><br>
<a href='#eda'>EDA</a><br>
- <a href='#null'>Null values</a><br>
- <a href='#channel'>Via which channel did user visited </a><br>
- <a href='#mobile'>Mobile users</a><br>
- <a href='#browser'>Browser based </a><br>
- <a href='#device'>Device Category</a><br>
- <a href='#os'>Operating system</a><br>
- <a href='#continent'>Continent Based</a><br>
- <a href='#metro'>Metro Based</a><br>
- <a href='#networkdomain'>Network Domain</a><br>
- <a href='#region'>Region</a><br>
- <a href='#country'>Country Based</a><br>
- <a href='#subcountry'>Sub Continent Based</a><br>
- <a href='#pVb'>Page view V/S bounces</a><br>
- <a href='#customers'>new cusotmer or old customer</a><br>
- <a href='#minmax'>Minmum & maximum revenu on daily basis</a><br>
- <a href='#month'>Revenue based on month</a><br>
- <a href='#day'>Revenue based on day</a><br>
- <a href='#weekday'>Revenue based on weekday</a><br>
- <a href='#adcontent'>Most Ad Content</a><br>
- <a href='#keywords'>Keywords used by users</a><br>
- <a href='#source'>Source from where users came</a><br>

# <a id='desc'> Description</a>

In this competition, you’re challenged to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. 

# <a id='about'>About notebook</a>

In this notebook we will look into the Dataset provided in the competetion and we will analyze the users of GStore.

# <a id='load_lib'>Load libraries</a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
import plotly.graph_objs as go
from plotly import tools
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
from plotly.tools import FigureFactory as ff
import random
from collections import Counter
import warnings
import json
import os
import datetime
from pandas.io.json import json_normalize
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
import pycountry
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

# <a id='load_data'>Load Dataset</a>

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
# train_df = pd.read_csv('flatten_train.csv')
# test_df = pd.read_csv('flatten_test.csv')

In [ ]:
# helper functions
def constant_cols(df):
    cols = []
    columns = df.columns.values
    for col in columns:
        if df[col].nunique(dropna = False) == 1:
            cols.append(col)
    return cols

def diff_cols(df1,df2):
    columns1 = df1.columns.values
    columns2 = df2.columns.values
    print(list(set(columns1) - set(columns2)))
    

def count_mean(col,color1,color2):
    col_count = train_df[col].value_counts()
    col_count_chart = go.Bar(x = col_count.head(10).index, y = col_count.head(10).values, name="Count",marker = dict(color=color1))

    col_mean_count = train_df[[col,'totals.transactionRevenue']][(train_df['totals.transactionRevenue'] >1)]
    col_mean_count = col_mean_count.groupby(col)['totals.transactionRevenue'].mean().sort_values(ascending=False)
    col_mean_count_chart = go.Bar(x = col_mean_count.head(10).index, y = col_mean_count.head(10).values, name="Mean",marker = dict(color=color2))

    fig = tools.make_subplots(rows = 1, cols = 2,subplot_titles=('Total Count','Mean Revenue'))
    fig.append_trace(col_count_chart, 1,1)
    fig.append_trace(col_mean_count_chart,1,2)
    py.iplot(fig)


In [ ]:
train.head()

# <a id='description'>Column Description</a><br>

- fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
- channelGrouping - The channel via which the user came to the Store.
- date - The date on which the user visited the Store.
- device - The specifications for the device used to access the Store.
- geoNetwork - This section contains information about the geography of the user.
- sessionId - A unique identifier for this visit to the store.
- socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
- totals - This section contains aggregate values across the session.
- trafficSource - This section contains information about the Traffic Source from which the session originated.
- visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
- visitNumber - The session number for this user. If this is the first session, then this is set to 1.
-visitStartTime - The timestamp (expressed as POSIX time)

Since few columns have json values lets convert flatten them

Source from where i got the code to flatten the json columns<br>
https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook

In [ ]:
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
train_df = load_df()

In [ ]:
train_df.head()

In [ ]:
test_df = load_df("../input/test.csv")

In [ ]:
# train_df.to_csv('flatten_train.csv')
# test_df.to_csv('flatten_test.csv')
diff_cols(train_df,test_df)

Since totals transaction Revenue is what we are going to predict.
and there is no campaignCode in test set

# <a id='cleaning'>Cleaning Dataset</a>

In [ ]:
train_constants = constant_cols(train_df)
test_constants = constant_cols(test_df)
print(train_constants)
print(test_constants)

In [ ]:
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].fillna(0)
train_df['date'] = train_df['date'].astype(str)
train_df["date"] = train_df["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
train_df["date"] = pd.to_datetime(train_df["date"])

## both the df has same cols with constant values lets remove them 

In [ ]:
train_constants = constant_cols(train_df)
test_constants = constant_cols(test_df)
train_df = train_df.drop(columns=train_constants,axis = 1)
test_df = test_df.drop(columns=test_constants, axis = 1)

# <a id='eda'>EDA</a><br>

# <a id='null'>Null values</a><br>

In [ ]:
null_values = train_df.isna().sum(axis = 0).reset_index()
null_values = null_values[null_values[0] > 50]
null_chart = [go.Bar(y = null_values['index'],x = null_values[0]*100/len(train_df), orientation = 'h')]
py.iplot(null_chart)

**Summary**<br>
- So many coloumns has null values<br>
- we will find why these columns are null and we will also see how we can manage them.

# <a id='channel'>Via which channel did user visited </a><br>

In [ ]:
data = train_df[['channelGrouping','totals.transactionRevenue']]
temp = data['channelGrouping'].value_counts()
chart = [go.Pie(labels = temp.index, values = temp.values)]
py.iplot(chart)

**Summary**<br>
-  Most of the users came via organic search.<br>
-   Paid search and affilate users are very less.

# <a id='mobile'>Mobile users</a><br>

In [ ]:
temp = train_df['device.isMobile'].value_counts()
chart = go.Bar(x = ["False","True"], y = temp.values)
py.iplot([chart])

**Summary**
- Many users browse the site from desktop or tablet

# <a id='browser'>Browser based</a><br>

In [ ]:
count_mean('device.browser',"#7FDBFF","#3D9970")

**Summary**
- The user visit count is very high for Chrome but Firefox users have genereted more revenue

# <a id='device'>Device Category</a><br>

In [ ]:
count_mean('device.deviceCategory',"#FF851B","#FF4136")

**Summary**
- Desktop site has generated more user count as well as more revenue
- One thing to note is tablet users have generated almost same revenue as mobile users 

# <a id='os'>Operating system</a><br>

In [ ]:
count_mean('device.operatingSystem',"#80DEEA","#0097A7")

**Summary**
- Less chrome OS users but high revenue is generated.
-For windows phone users also generated good revenue.

# <a id='continent'>Continent Based</a><br>

In [ ]:
count_mean('geoNetwork.continent',"#F48FB1","#C2185B")

**Summary**
- African Users have generated more than a billion mean revenue.
- Users from american have used the website a lot but didnt purchased products.

# <a id='country'>Country Based</a><br>

In [ ]:
data = train_df[['geoNetwork.country','totals.transactionRevenue']][(train_df['totals.transactionRevenue'] >1)]
temp = data.groupby('geoNetwork.country',as_index=False)['totals.transactionRevenue'].mean()
temp['code'] = 'sample'
for i,country in enumerate(temp['geoNetwork.country']):
    mapping = {country.name: country.alpha_3 for country in pycountry.countries}
    temp.set_value(i,'code',mapping.get(country))
chart = [ dict(
        type = 'choropleth',
        locations = temp['code'],
        z = temp['totals.transactionRevenue'],
        text = temp['geoNetwork.country'],
        autocolorscale = True,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                  width = 0.5
            ) ),
        colorbar = dict(
            autotick = True,
            title = 'Mean Revenue'),
      ) ]

layout = dict(
    title = 'Mean revenue based on country',
    geo = dict(
        showframe = True,
        showcoastlines = True,
         showocean = True,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=chart, layout=layout )
py.iplot( fig, validate=False)

**Summary**
- We can see the revenue generated based on the countries
- only 4 countries in africa have generated huge mean revenue.

# <a id='metro'>Metro Based</a><br>

In [ ]:
count_mean('geoNetwork.metro',"#CE93D8", "#7B1FA2")

**Summary**
- Most of the users metro location is not present in the dataset.
- may be we have to assume some hypothesis here

# <a id='networkdomain'>Network Domain</a><br>

In [ ]:
count_mean('geoNetwork.networkDomain','#90CAF9','#1976D2')

**Summary**
- Most users who generate revenue uses digitalwest.net
- Again same we have huge users count from unknown source/ (not set)

# <a id='region'>Region</a><br>

In [ ]:
count_mean('geoNetwork.region','#DCE775','#AFB42B')

** Summary**
- Tokyo has generated more than 1 Billion mean revenue

# <a id='subcountry'>Sub Continent Based</a><br>

In [ ]:
count_mean('geoNetwork.subContinent','#FFE082','#FFA000')

**Summary**
- No surprise Africa stands top in mean revenue 

# <a id='pVb'>Page view V/S bounces</a><br>

In [ ]:
train_df['totals.pageviews'] = train_df['totals.pageviews'].fillna(0).astype('int32')
train_df['totals.bounces'] = train_df['totals.bounces'].fillna(0).astype('int32')

pageview = train_df.groupby('date')['totals.pageviews'].apply(lambda x:x[x >= 1].count()).reset_index()
bounce = train_df.groupby('date')['totals.bounces'].apply(lambda x:x[x >= 1].count()).reset_index()

pageviews = go.Scatter(x = pageview['date'],y= pageview['totals.pageviews'], name = 'Pageview',marker=dict(color = "#B0BEC5"))


bounces = go.Scatter(x = bounce['date'],y= bounce['totals.bounces'],name = 'Bounce',marker=dict(color = "#37474F"))

py.iplot([pageviews,bounces])

**Summary**
- We can see the based on pageview we have increase/decrease of bounce rate 

# <a id='customers'>new cusotmer or old customer</a><br>

In [ ]:
train_df['totals.newVisits'] = train_df['totals.newVisits'].fillna(0).astype('int32')
train_df['totals.hits'] = train_df['totals.hits'].fillna(0).astype('int32')

newvisit = train_df.groupby('date')['totals.newVisits'].apply(lambda x:x[x == 1].count()).reset_index()
oldVisit = train_df.groupby('date')['totals.newVisits'].apply(lambda x:x[x == 0].count()).reset_index()
hit = train_df.groupby('date')['totals.hits'].apply(lambda x:x[x >= 1].count()).reset_index()


hits = go.Scatter(x = hit['date'],y = hit['totals.hits'], name = 'total hits', marker=dict(color = '#FFEE58'))

new_vist = go.Scatter(x = newvisit['date'],y= newvisit['totals.newVisits'],name = 'New Vists', marker=dict(color = '#F57F17'))

oldvisit = go.Scatter(x = oldVisit['date'],y = oldVisit['totals.newVisits'], name = 'Old Visit', marker=dict(color = '#FFD600'))

py.iplot([hits, new_vist, oldvisit])

** Summary **
- Out of all the hits we have more new visit than old visit
- That means the returning customer is very less than the new customers.
- Or there can be other meaning.

# <a id='minmax'>Minmum & maximum revenue on daily basis</a><br>

In [ ]:
temp = train_df[(train_df['totals.transactionRevenue'] >0)]
data = temp[['totals.transactionRevenue','date']].groupby('date')['totals.transactionRevenue'].agg(['min','max']).reset_index()
mean = go.Scatter(x = data['date'], y = data['min'],name = "Min",marker = dict(color = '#00E676'))
count = go.Scatter(x = data['date'],y = data['max'], name = "Max",marker = dict(color = '#00838F'))
py.iplot([mean,count])

** Summary **
- I have removed all the non-zero transaction.
- This graph is to understand the minimum and maximum revenue the company generates.
- 5th april the company generated the maximum revenue.
- there are few days where the maximum and minimum revenue are same(Eg - 15 Jan 2017)

# <a id='month'>Revenue based on month</a><br>

In [ ]:
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['weekday'] = train_df['date'].dt.weekday

In [ ]:
temp = train_df.groupby('month')['totals.transactionRevenue'].agg(['count','mean']).reset_index()
count_chart = go.Bar(x = temp['month'], y = temp['count'],name = 'Count',marker = dict(color = "#E6EE9C"))
mean_chart = go.Bar(x = temp['month'],y = temp['mean'], name = 'Mean',marker = dict(color = "#AFB42B"))

fig = tools.make_subplots(rows = 1, cols = 2, subplot_titles = ('Total Count', 'Mean Count'))
fig.append_trace(count_chart,1,1)
fig.append_trace(mean_chart, 1,2)
py.iplot(fig)

** Summary **
- It is seen that November month has higest number of visitors but the transaction generated in that month is very low.
- April month has generated higest mean revenue while its visitors are not high.

# <a id='day'>Revenue based on day</a><br>

In [ ]:
temp = train_df.groupby('day')['totals.transactionRevenue'].agg(['count','mean']).reset_index()
count_chart = go.Bar(x = temp['day'], y = temp['count'],name = 'Count', marker = dict(color = '#1DE9B6'))
mean_chart = go.Bar(x = temp['day'],y = temp['mean'], name = 'Mean', marker = dict(color = '#00796B'))

fig = tools.make_subplots(rows = 1, cols = 2, subplot_titles = ('Total Count', 'Mean Count'))
fig.append_trace(count_chart,1,1)
fig.append_trace(mean_chart, 1,2)
py.iplot(fig)

** Summary **
- It is seen that on 31th day the view count is very less I dont blame any because only Jan, Mar, May, Jul, Aug, Oct, Dec has 31 days.
- But the intresting fact is that the 2nd highest mean revenue is on day 31st.
- I think it can either be because of Jan or Dec as they are the start or end of the year.

# <a id='weekday'>Revenue based on weekday</a><br>

In [ ]:
temp = train_df.groupby('weekday')['totals.transactionRevenue'].agg(['count','mean']).reset_index()
count_chart = go.Bar(x = temp['weekday'], y = temp['count'],name = 'Count', marker = dict(color = '#9575CD'))
mean_chart = go.Bar(x = temp['weekday'],y = temp['mean'], name = 'Mean', marker = dict(color = '#B388FF'))

fig = tools.make_subplots(rows = 1, cols = 2, subplot_titles = ('Total Count', 'Mean Count'))
fig.append_trace(count_chart,1,1)
fig.append_trace(mean_chart, 1,2)
py.iplot(fig)

** Summary **
- Most of the revenue is generated on monday.
- very less revenue is generated on weekend.

# <a id='adcontent'>Most Ad Content</a><br>

In [ ]:
train_df['trafficSource.adContent'] = train_df['trafficSource.adContent'].fillna('')
wordcloud2 = WordCloud(width=800, height=400).generate(' '.join(train_df['trafficSource.adContent']))
plt.figure( figsize=(15,20))
plt.imshow(wordcloud2)
plt.axis("off")
plt.show()

** Summary **
- Image speak for it self

# <a id='keywords'>Keywords used by users</a><br>

In [ ]:
train_df['trafficSource.keyword'] = train_df['trafficSource.keyword'].fillna('')
wordcloud2 = WordCloud(width=800, height=400).generate(' '.join(train_df['trafficSource.keyword']))
plt.figure( figsize=(20,20) )
plt.imshow(wordcloud2)
plt.axis("off")
plt.show()

<a id='source'>Source from where users came</a><br>

In [ ]:
train_df['trafficSource.source'] = train_df['trafficSource.source'].fillna('')
wordcloud2 = WordCloud(width=800, height=400).generate(' '.join(train_df['trafficSource.source']))
plt.figure( figsize=(15,20) )
plt.imshow(wordcloud2)
plt.axis("off")
plt.show()

# Glad that you made it till end.
# Please Upvote to boost me :-).